In [1]:
#Install Packages
!pip install -q faiss-cpu
!pip install -q sentence-transformers

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Fake review detection

In [3]:
review_tokenizer = AutoTokenizer.from_pretrained("astrosbd/fake-reviews-distilbert")
review_model = AutoModelForSequenceClassification.from_pretrained("astrosbd/fake-reviews-distilbert")
def detect_fake_reviews(review):
    review_inputs = review_tokenizer(review, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        review_outputs = review_model(**review_inputs)
    
    # Get the predicted class (0 = genuine, 1 = fake)
    review_predictions = torch.argmax(review_outputs.logits, dim=-1)
    if review_predictions == 0:
        label = 1
    else:
        label = 0
    
    return label

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# Sentimental Analysis

In [4]:
sentiment_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

def analyze_sentiment(text):
    sentiment_inputs = sentiment_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        sentiment_outputs = sentiment_model(**sentiment_inputs)
    sentiment_logits = sentiment_outputs.logits
    sentiment_probs = torch.nn.functional.softmax(sentiment_logits, dim=-1)
    sentiment_predicted_class = torch.argmax(sentiment_probs, dim=-1).item()
    if sentiment_predicted_class == 2:
        label = 1
    elif sentiment_predicted_class == 1:
        label = 0
    else:
        label = -1
    
    return label


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# pd.merge(s, u, on="A", how="inner")

In [6]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

2024-08-26 03:40:22.058018: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-26 03:40:22.058128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-26 03:40:22.195324: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
product_df = pd.read_csv('/kaggle/input/amazon-data/products.csv', encoding='ISO-8859-1')
review_df = pd.read_csv('/kaggle/input/amazon-data/reviews.csv', encoding='ISO-8859-1')

In [8]:
product_df.head()

,Product ID,Product Name,Product Description,Rating
0,B071Z8M4KX,boAt BassHeads 100 in-Ear Wired Headphones wit...,The perfect way to add some style and stand ou...,4.1
1,\tB09XS7JWHH,Sony WH-1000XM5 Best Active Noise Cancelling W...,Industry Leading noise cancellation-two proces...,4.5
2,B0D5YCYS1G,"OnePlus Nord CE4 Lite 5G (Super Silver, 8GB RA...",5500 mAh Battery & Reverse Charging: Ditch the...,4.0
3,B09N3ZNHTY,boAt Airdopes 141 Bluetooth Truly Wireless in ...,Playback- Enjoy an extended break on weekends ...,3.8
4,B0B4N77Y34,"HP Laptop 15s, AMD Ryzen 5 5500U, 15.6-inch (3...",?6-core AMD Ryzen 5 5500U?Harness efficient mu...,4.1


In [9]:
product_df['Product ID'] = product_df['Product ID'].str.strip()
product_df['Product ID'] = product_df['Product ID'].str.replace(r'\s+', '', regex=True)

In [10]:
product_df.rename(columns = {'Product ID': 'Id', 'Product Name': 'name', 'Product Description': 'description', 'Rating': 'rating'}, inplace = True)

In [11]:
product_df

,Id,name,description,rating
0,B071Z8M4KX,boAt BassHeads 100 in-Ear Wired Headphones wit...,The perfect way to add some style and stand ou...,4.1
1,B09XS7JWHH,Sony WH-1000XM5 Best Active Noise Cancelling W...,Industry Leading noise cancellation-two proces...,4.5
2,B0D5YCYS1G,"OnePlus Nord CE4 Lite 5G (Super Silver, 8GB RA...",5500 mAh Battery & Reverse Charging: Ditch the...,4.0
3,B09N3ZNHTY,boAt Airdopes 141 Bluetooth Truly Wireless in ...,Playback- Enjoy an extended break on weekends ...,3.8
4,B0B4N77Y34,"HP Laptop 15s, AMD Ryzen 5 5500U, 15.6-inch (3...",?6-core AMD Ryzen 5 5500U?Harness efficient mu...,4.1
5,B098JYT4SY,"ZEBRONICS ZEB-JAGUAR Wireless Mouse, 2.4GHz wi...","Enjoy seamless connectivity with 2.4GHz, Plug ...",4.0
6,B0BXWZWSVK,cimetech KF10 EasyTyping Keyboard Mouse Combo ...,?Easy Typing?KF10 Keyboard is designed to make...,3.8
7,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Regular Fit Half Sleeve Polo is made of comfor...,3.9
8,B0C6GQWGDV,Leriya Fashion Western Dress || Rayon Solid Bu...,Leriya Fashion is the ultimate destination for...,3.2
9,B077BFH786,SOFTSPUN Microfiber Cloth - 4 pcs - 40x40 cms ...,"SUPER ABSORBENT, LINT SCRATCH FREE MultiPurpos...",4.3


In [12]:
product_descriptions = product_df['description'].tolist()

In [13]:
review_df

,Product ID,Product Name,Product Review,Review Rating
0,B071Z8M4KX,boAt BassHeads 100 in-Ear Wired Headphones wit...,Build quality is good but sound quality is dec...,3
1,B071Z8M4KX,boAt BassHeads 100 in-Ear Wired Headphones wit...,Nice product to buy if anyone looking in this ...,5
2,B071Z8M4KX,boAt BassHeads 100 in-Ear Wired Headphones wit...,The sound quality is awesome. It hurts a bit o...,5
3,B071Z8M4KX,boAt BassHeads 100 in-Ear Wired Headphones wit...,The product is working fine and good even afte...,4
4,B071Z8M4KX,boAt BassHeads 100 in-Ear Wired Headphones wit...,I don't even remember when I bought it. But I ...,5
...,...,...,...,...
155,B01N0AO8RI,Continental Coffee Xtra Instant Coffee Powder ...,It's a good coffee for daily usage if you love...,3
156,B01N0AO8RI,Continental Coffee Xtra Instant Coffee Powder ...,Excellent brand of coffee,4
157,B01N0AO8RI,Continental Coffee Xtra Instant Coffee Powder ...,A good strong brew coffee should have a bold f...,5
158,B01N0AO8RI,Continental Coffee Xtra Instant Coffee Powder ...,"Coming from nescafe, this taste stands out!",4


In [14]:
review_df.rename(columns = {'Product ID': 'Id', 'Product Name': 'name', 'Product Review': 'description', 'Review Rating': 'rating'}, inplace = True)

In [15]:
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
vectors = encoder.encode(product_descriptions)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
vectors.shape

(16, 384)

In [17]:
dim = vectors.shape[1]
dim

384

In [18]:
index = faiss.IndexFlatL2(dim)
index.add(vectors)
index

<faiss.swigfaiss_avx512.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f8bb8d66130> >

In [19]:
des = """
Regular Fit Half Sleeve Polo is made of comfortable, Bio Washed cotton-poly pique fabric, a three-button
placket, and ribbed cuffs for a classic look. Fabric Composition - Cotton 60% Poly 40% Blend, Bio Wash Pique 
Fabric. Pattern - Solid Color Men\x92s Polo Tshirt, Rib Collar & Sleeve for Amazing Fit. Classic "American Crew" 
Signature Polo with Logo Embroidery on Chest. \x93Made In India\x94 by Socially Compliant MSME Factory. All 
Components Used to make this T-Shirt are Proudly "Made in India".
Product Color May Slightly Vary Due To Photographic Lighting Sources Or Your Monitor Settings.
"""
vec = encoder.encode(des)
vec.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(384,)

In [20]:
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 384)

In [21]:
distances, I = index.search(svec, k=5)
print(distances)
print(I)

[[3.4476783e-13 1.2927146e+00 1.3313311e+00 1.3448181e+00 1.3567086e+00]]
[[ 7  8 10 12 11]]


In [22]:
row_idx = I[0]
row_idx

array([ 7,  8, 10, 12, 11])

In [23]:
recommend_df = product_df.iloc[row_idx]

In [24]:
new_df = pd.merge(recommend_df, review_df, on="Id", how="inner")

In [25]:
new_df.head()

,Id,name_x,description_x,rating_x,name_y,description_y,rating_y
0,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Regular Fit Half Sleeve Polo is made of comfor...,3.9,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,"Good fabrics , nice fitting\nValue for money",5
1,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Regular Fit Half Sleeve Polo is made of comfor...,3.9,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Nice colour as shown in image and material is ...,5
2,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Regular Fit Half Sleeve Polo is made of comfor...,3.9,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Colour and fitting are perfect.,5
3,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Regular Fit Half Sleeve Polo is made of comfor...,3.9,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Got this delivered yesterday. I like the overa...,5
4,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Regular Fit Half Sleeve Polo is made of comfor...,3.9,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,First time I purchased t shirts in Amazon and ...,5


In [26]:
new_df.drop(columns=['name_y', 'rating_x', 'description_x'], inplace = True)

In [27]:
new_df

,Id,name_x,description_y,rating_y
0,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,"Good fabrics , nice fitting\nValue for money",5
1,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Nice colour as shown in image and material is ...,5
2,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Colour and fitting are perfect.,5
3,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Got this delivered yesterday. I like the overa...,5
4,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,First time I purchased t shirts in Amazon and ...,5
5,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,best,5
6,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,nice.,5
7,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Good quality,5
8,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,as per mentioned size was not perfect . so don...,1
9,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Product quality is so cheap ...it is totally w...,1


In [28]:
new_df['detect_review'] = new_df['description_y'].apply(detect_fake_reviews)

In [29]:
# new_df = new_df[new_df['detect_review'] != -1]

In [30]:
new_df

,Id,name_x,description_y,rating_y,detect_review
0,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,"Good fabrics , nice fitting\nValue for money",5,0
1,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Nice colour as shown in image and material is ...,5,0
2,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Colour and fitting are perfect.,5,0
3,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Got this delivered yesterday. I like the overa...,5,0
4,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,First time I purchased t shirts in Amazon and ...,5,0
5,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,best,5,1
6,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,nice.,5,1
7,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Good quality,5,1
8,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,as per mentioned size was not perfect . so don...,1,0
9,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Product quality is so cheap ...it is totally w...,1,0


In [31]:
def weight_review(value):
    if value == 1:
        return 1 * 0.5

In [32]:
new_df['weight'] = new_df['detect_review'].apply(weight_review)

In [33]:
new_df.drop(columns=['detect_review'], inplace = True)

In [34]:
new_df

,Id,name_x,description_y,rating_y,weight
0,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,"Good fabrics , nice fitting\nValue for money",5,NaN
1,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Nice colour as shown in image and material is ...,5,NaN
2,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Colour and fitting are perfect.,5,NaN
3,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Got this delivered yesterday. I like the overa...,5,NaN
4,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,First time I purchased t shirts in Amazon and ...,5,NaN
5,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,best,5,0.5
6,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,nice.,5,0.5
7,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Good quality,5,0.5
8,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,as per mentioned size was not perfect . so don...,1,NaN
9,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Product quality is so cheap ...it is totally w...,1,NaN


In [35]:
new_df = new_df.dropna(subset=['weight'])

In [36]:
new_df

,Id,name_x,description_y,rating_y,weight
5,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,best,5,0.5
6,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,nice.,5,0.5
7,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Good quality,5,0.5
19,B0C6GQWGDV,Leriya Fashion Western Dress || Rayon Solid Bu...,Good quality value of money,5,0.5
21,B0B5HL3VRF,Reebok Mens Stride Runner Running Shoe,Very lightweight and comfortable. Looks very e...,4,0.5
32,B01MTQ5M7B,ASIAN Men's Wonder-13 Sports Running Shoes,Comfortable,4,0.5


In [37]:
def norm_rating(value):
    norm_value = (value - 1)/4
    
    return norm_value

In [38]:
new_df['norm_rate'] = new_df['rating_y'].apply(norm_rating)
new_df.drop(columns=['rating_y'], inplace = True)
new_df

/tmp/ipykernel_24/2692309666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['norm_rate'] = new_df['rating_y'].apply(norm_rating)
/tmp/ipykernel_24/2692309666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(columns=['rating_y'], inplace = True)


,Id,name_x,description_y,weight,norm_rate
5,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,best,0.5,1.00
6,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,nice.,0.5,1.00
7,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Good quality,0.5,1.00
19,B0C6GQWGDV,Leriya Fashion Western Dress || Rayon Solid Bu...,Good quality value of money,0.5,1.00
21,B0B5HL3VRF,Reebok Mens Stride Runner Running Shoe,Very lightweight and comfortable. Looks very e...,0.5,0.75
32,B01MTQ5M7B,ASIAN Men's Wonder-13 Sports Running Shoes,Comfortable,0.5,0.75


In [39]:
new_df['norm_rate'] = new_df['norm_rate'] * 0.5

/tmp/ipykernel_24/1143191119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['norm_rate'] = new_df['norm_rate'] * 0.5


In [40]:
new_df

,Id,name_x,description_y,weight,norm_rate
5,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,best,0.5,0.500
6,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,nice.,0.5,0.500
7,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,Good quality,0.5,0.500
19,B0C6GQWGDV,Leriya Fashion Western Dress || Rayon Solid Bu...,Good quality value of money,0.5,0.500
21,B0B5HL3VRF,Reebok Mens Stride Runner Running Shoe,Very lightweight and comfortable. Looks very e...,0.5,0.375
32,B01MTQ5M7B,ASIAN Men's Wonder-13 Sports Running Shoes,Comfortable,0.5,0.375


In [41]:
new_df['sentiment'] = new_df['description_y'].apply(detect_fake_reviews)

/tmp/ipykernel_24/1857815416.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['sentiment'] = new_df['description_y'].apply(detect_fake_reviews)


In [42]:
new_df = new_df[new_df['sentiment'] != -1]

In [43]:
def score_sentiment(value):
    if value == 1:
        return 1 * 0.7
    else:
        return 0.5

In [44]:
new_df['score_weight'] = new_df['sentiment'].apply(score_sentiment)
new_df.drop(columns=['sentiment'], inplace = True)

In [45]:
new_df.drop(columns=['description_y'], inplace = True)

In [46]:
# new_df['norm_rate'] = new_df['norm_rate'] * 0.5

In [47]:
new_df

,Id,name_x,weight,norm_rate,score_weight
5,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,0.5,0.500,0.7
6,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,0.5,0.500,0.7
7,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,0.5,0.500,0.7
19,B0C6GQWGDV,Leriya Fashion Western Dress || Rayon Solid Bu...,0.5,0.500,0.7
21,B0B5HL3VRF,Reebok Mens Stride Runner Running Shoe,0.5,0.375,0.7
32,B01MTQ5M7B,ASIAN Men's Wonder-13 Sports Running Shoes,0.5,0.375,0.7


In [48]:
new_df = new_df.groupby('Id').agg({
    'weight': 'sum', 
    'norm_rate': 'sum',
    'score_weight': 'sum',
    'Id': 'first',
    'name_x': 'first'
})

In [49]:
new_df['score'] = new_df['weight'] + new_df['norm_rate'] + new_df['score_weight']

In [50]:
new_df

,weight,norm_rate,score_weight,Id,name_x,score
Id,,,,,,
B0168Q5U7C,1.5,1.500,2.1,B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,5.100
B01MTQ5M7B,0.5,0.375,0.7,B01MTQ5M7B,ASIAN Men's Wonder-13 Sports Running Shoes,1.575
B0B5HL3VRF,0.5,0.375,0.7,B0B5HL3VRF,Reebok Mens Stride Runner Running Shoe,1.575
B0C6GQWGDV,0.5,0.500,0.7,B0C6GQWGDV,Leriya Fashion Western Dress || Rayon Solid Bu...,1.700


In [51]:
new_df.drop(columns=['weight', 'norm_rate', 'score_weight'], inplace = True)

In [52]:
new_df.drop(columns=['Id'], inplace = True)

In [53]:
new_df

,name_x,score
Id,,
B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,5.100
B01MTQ5M7B,ASIAN Men's Wonder-13 Sports Running Shoes,1.575
B0B5HL3VRF,Reebok Mens Stride Runner Running Shoe,1.575
B0C6GQWGDV,Leriya Fashion Western Dress || Rayon Solid Bu...,1.700


In [54]:
new_df = new_df.sort_values(by='score', ascending=False)
new_df

,name_x,score
Id,,
B0168Q5U7C,AMERICAN CREW CottonPoly Solid Polo T-Shirt wi...,5.100
B0C6GQWGDV,Leriya Fashion Western Dress || Rayon Solid Bu...,1.700
B01MTQ5M7B,ASIAN Men's Wonder-13 Sports Running Shoes,1.575
B0B5HL3VRF,Reebok Mens Stride Runner Running Shoe,1.575


In [55]:
for idx, row in new_df.iterrows():
    print(idx)
    print(f"Name of product: {row['name_x']}")
    print('----------------------------------')

B0168Q5U7C
Name of product: AMERICAN CREW CottonPoly Solid Polo T-Shirt with No.3 Applique for Men
----------------------------------
B0C6GQWGDV
Name of product: Leriya Fashion Western Dress || Rayon Solid Button Front Shirt Dress for Women || Roll Tab Sleeve & Collared Neck Flared Dress || Office || Summer Short Dresses for Women.
----------------------------------
B01MTQ5M7B
Name of product: ASIAN Men's Wonder-13 Sports Running Shoes
----------------------------------
B0B5HL3VRF
Name of product: Reebok Mens Stride Runner Running Shoe
----------------------------------


In [56]:
def precision_at_k(recommended_items, relevant_items, k):
    recommended_items_at_k = recommended_items[:k]
    relevant_recommended = set(recommended_items_at_k) & set(relevant_items)
    return len(relevant_recommended) / k

# Example usage
recommended_items = ['AMERICAN CREW CottonPoly Solid Polo T-Shirt with No.3 Applique for Men', 'Leriya Fashion Western Dress || Rayon Solid Button Front Shirt Dress for Women || Roll Tab Sleeve & Collared Neck Flared Dress || Office || Summer Short Dresses for Women.', 'ASIAN Mens Wonder-13 Sports Running Shoes', 'Reebok Mens Stride Runner Running Shoe']
relevant_items = ['AMERICAN CREW CottonPoly Solid Polo T-Shirt with No.3 Applique for Men', 'Leriya Fashion Western Dress || Rayon Solid Button Front Shirt Dress for Women || Roll Tab Sleeve & Collared Neck Flared Dress || Office || Summer Short Dresses for Women.', 'SOFTSPUN Microfiber Cloth - 4 pcs - 40x40 cms - 340 GSM Grey! Thick Lint & Streak-Free Multipurpose Cloths - Automotive Microfibre Towels for Car Bike Cleaning Polishing Washing & Detailing.\x85','Reebok Mens Stride Runner Running Shoe']
k = 4

precision = precision_at_k(recommended_items, relevant_items, k)
print(f"Precision@{k}: {precision:.4f}")

Precision@4: 0.7500
